In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn
import pandas as pd
import numpy as np

from sys import float_info
from line_profiler import LineProfiler
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.datasets import make_blobs

seaborn.set()
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)


%load_ext Cython

In [2]:
def profile_print(func_to_call, *args):
    profiler = LineProfiler()
    profiler.add_function(func_to_call)
    profiler.runcall(func_to_call, *args)
    profiler.print_stats()
    
def profile_print_go_deeper(func_to_call, func_in_func, *args):
    profiler = LineProfiler()
    profiler.add_function(func_to_call)
    profiler.add_function(func_in_func)
    profiler.runcall(func_to_call, *args)
    profiler.print_stats()

In [3]:
df_sns = pd.read_csv('snsdata.csv', sep=',')
df_sns.dropna(axis=0, inplace=True)
sns_val = df_sns.drop(['gradyear', 'gender', 'age', 'friends'], axis=1).values
sns_val_norm = (sns_val - sns_val.mean(axis=0)) / sns_val.std(axis=0)

In [4]:
import math
from sklearn.base import ClusterMixin
import random

class Kmeans_Numpy(BaseEstimator, ClusterMixin): 
    
    def __init__(self, k=2, metric='euclidean', max_iter=1000,
                 random_state=0, init='random', epsilon=0.00001):
        """
        Инициализация метода
        :k - количество кластеров
        :metric - функция расстояния между объектами
        :max_iter - максиальное количество итераций
        :random_state - seed для инициализации генератора случайных чисел
        """
        
        self.k = k
        self.random_state = random_state
        self.metric = metric
        self.max_iter = max_iter
        self.init = init
        self.dev = sys.float_info.max
        self.epsilon = epsilon
        return None

    def init_centroids(self, X):
        # Инициализируем массив с центроидами кластеров
        if self.init == 'random':
            self.centroids = X[np.random.permutation(xrange(0, X.shape[0]))[:self.k]]
        
    def fit(self, X, y=None):
        """
        Процедура обучения k-means
        """
        
        # Инициализация генератора случайных чисел
        np.random.seed(self.random_state)
        
        # Массив с метками кластеров для каждого объекта из X
        self.labels = np.empty(X.shape[0], dtype=np.int64)
        
        # Инициализируем центроиды
        self.init_centroids(X)
        
        for i in xrange(1, self.max_iter):
            # Пересчитали принадлежность
            self.labels = self.predict(X)
            # Пересчитали центроиды
            for x in xrange(0, self.k):
                tmp_lab = self.labels == x
                if X[tmp_lab].shape[0]:
                    self.centroids[x] = X[self.labels == x].mean(0)
            
            # Посчитали cумму отклонений
            new_dev = 0;
            for x in xrange(0, self.k):
                tmp_lab = self.labels == x
                if X[tmp_lab].shape[0]:
                    new_dev += np.apply_along_axis(np.linalg.norm, 0, (X[tmp_lab] - self.centroids[x])).sum()
            if abs(self.dev - new_dev) < self.epsilon:
                break;
            self.dev = new_dev
        return self

    def predict(self, X, y=None):
        """
        Процедура предсказания кластера
        
        Возвращает метку ближайшего кластера для каждого объекта
        """
        return ((self.centroids[:, np.newaxis, :] - X)**2).sum(2).argmin(0)

In [5]:
kluster_num = 9
random_seed = 352245
kmeans_rnd_seed = 14124
samples_num = 10000

In [6]:
## Работоспособность KMeans (python + numpy)
sns_kmeans = Kmeans_Numpy(k=kluster_num, random_state=kmeans_rnd_seed)

In [7]:
%%timeit
sns_kmeans.fit(sns_val_norm)

1 loop, best of 3: 4.67 s per loop


In [8]:
profile_print_go_deeper(sns_kmeans.fit, sns_kmeans.predict, sns_val_norm)

Timer unit: 1e-06 s

Total time: 5.28773 s
File: <ipython-input-4-87dbf998016b>
Function: fit at line 31

Line #      Hits         Time  Per Hit   % Time  Line Contents
    31                                               def fit(self, X, y=None):
    32                                                   """
    33                                                   Процедура обучения k-means
    34                                                   """
    35                                                   
    36                                                   # Инициализация генератора случайных чисел
    37         1           16     16.0      0.0          np.random.seed(self.random_state)
    38                                                   
    39                                                   # Массив с метками кластеров для каждого объекта из X
    40         1           16     16.0      0.0          self.labels = np.empty(X.shape[0], dtype=np.int64)
    41              

In [9]:
df_sns['class'] = sns_kmeans.labels
df_sns[['class', 'gender', 'age']].groupby(['class', 'gender']).aggregate('count').transpose()

class     0       1        2         3         4        5         6       \
gender    F  M    F   M    F    M    F    M    F   M    F   M     F    M   
age     500  4  305  78  169  274  600  120  619  69  876  18  3596  228   

class       7          8       
gender      F     M    F    M  
age     12242  3777  404  126

In [10]:
for i in range(0, 9):
    print i, (sns_kmeans.labels == i).sum()
F_kmeans = df_sns.drop(['gradyear', 'gender', 'age', 'friends', 'class'], axis=1)
F_kmeans = pd.DataFrame(columns=F_kmeans.columns, data=sns_kmeans.centroids).transpose()
for i in range(0, 9):
    print (F_kmeans.loc[:,i]).sort_values()[-5:]

0 504
1 383
2 443
3 720
4 688
5 894
6 3824
7 16019
8 530
hot             0.199109
mall            0.220395
football        0.299542
shopping        0.485949
cheerleading    5.510765
Name: 0, dtype: float64
death     0.275681
church    1.217691
god       2.253207
jesus     2.369786
bible     6.153734
Name: 1, dtype: float64
rock          0.627325
sports        0.682258
basketball    0.763690
football      1.340572
baseball      5.492383
Name: 2, dtype: float64
drunk     1.939984
sex       2.138034
hair      2.605652
drugs     2.957622
kissed    3.168000
Name: 3, dtype: float64
clothes        0.554469
mall           0.623619
shopping       0.763631
abercrombie    3.917823
hollister      4.115273
Name: 4, dtype: float64
football      0.243619
sports        0.779573
basketball    0.943376
volleyball    2.674364
softball      3.056029
Name: 5, dtype: float64
dress       0.597157
mall        0.628572
dance       0.641483
cute        0.693184
shopping    0.837276
Name: 6, dtype: float64
death

In [11]:
%%cython -a
import numpy as np
cimport numpy as np
import cython

cdef double double_max = 1e100

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.initializedcheck(False)
@cython.optimize.unpack_method_calls(True)
cdef class Kmeans_Cython: 
    
    cdef long k;
    cdef long max_iter;
    cdef long random_state;
    cdef long[:] labels;
    cdef str metric;
    cdef str init;
    cdef double dev;
    cdef double epsilon;
    cdef double[:,:] centroids;
    
    def __init__(self, 
                 long k=2,
                 str metric='euclidean',
                 long max_iter=1000, 
                 long random_state=0,
                 str init='random',
                 double epsilon=0.00001):
        """
        Инициализация метода
        :k - количество кластеров
        :metric - функция расстояния между объектами
        :max_iter - максиальное количество итераций
        :random_state - seed для инициализации генератора случайных чисел
        """
        
        self.k = k
        self.random_state = random_state
        self.metric = metric
        self.max_iter = max_iter
        self.init = init
        self.dev = double_max
        self.epsilon = epsilon

    cpdef init_centroids(self, double[:,:] X):
        # Инициализируем массив с центроидами кластеров
        cdef long[:] indices;
        self.centroids = np.zeros([self.k, X.shape[1]])
        if self.init == 'random':
            indices = np.random.permutation(range(0, X.shape[0]))[:self.k]
            for i in range(0, self.k):
                for j in range(0, X.shape[1]):
                    self.centroids[i, j] = X[indices[i], j]
        
        
    cpdef fit(self, double[:,:] X):
        """
        Процедура обучения k-means
        """
        
        cdef double tmp_dev;
        cdef double new_dev;
        cdef long[:] count = np.zeros(self.k, dtype=long);
        
        self.labels = np.zeros(X.shape[0], dtype=long)
        
        # Инициализация генератора случайных чисел
        np.random.seed(self.random_state)
        
        # Массив с метками кластеров для каждого объекта из X
        
        # Инициализируем центроиды
        self.init_centroids(X)
        
        for it in range(self.max_iter):
            # Пересчитали принадлежность
            self.labels = self.predict(X)
            # Пересчитали центроиды
            for i in range(self.k):
                count[i] = 0
                for j in range(X.shape[1]):
                    self.centroids[i, j] = 0
            for i in range(X.shape[0]):
                count[self.labels[i]] += 1
                for j in range(X.shape[1]):
                    self.centroids[self.labels[i], j] += X[i, j]
            for i in range(self.k):
                if count[i]:
                    for j in range(X.shape[1]):
                        self.centroids[i, j] /= count[i]
            
            # Посчитали cумму отклонений
            new_dev = 0
            for x in range(X.shape[0]):
                tmp_dev = 0
                for j in range(X.shape[1]):
                    tmp_dev += (X[x, j] - self.centroids[self.labels[x], j]) ** 2
                new_dev += tmp_dev**(1.0/2)
            if abs(self.dev - new_dev) < self.epsilon:
                break;
            self.dev = new_dev

    cpdef long[:] predict(self, double[:,:] X):
        """
        Процедура предсказания кластера
        
        Возвращает метку ближайшего кластера для каждого объекта
        """
        
        cdef long[:] tmp_lab = np.zeros(X.shape[0], dtype=long);
        cdef double[:] tmp_dist = np.zeros(X.shape[0], dtype=float);
        cdef double dist;
        
        for i in range(X.shape[0]):
            tmp_dist[i] = double_max
        
        for i in range(X.shape[0]):
            for j in range(self.k):
                dist = 0
                for z in range(X.shape[1]):
                    dist += (X[i, z] - self.centroids[j, z]) ** 2
                if dist < tmp_dist[i]:
                    tmp_lab[i] = j;
                    tmp_dist[i] = dist
        return tmp_lab
    
    cpdef double[:,:] get_centroids(self):
        return self.centroids

In [12]:
## Работоспособность KMeansCython
sns_kmeans_cython = Kmeans_Cython(k=kluster_num, random_state=kmeans_rnd_seed)

In [13]:
%%timeit
sns_kmeans_cython.fit(sns_val_norm)

1 loop, best of 3: 1.41 s per loop


In [14]:
df_sns['class'] = sns_kmeans_cython.predict(sns_val_norm)
df_sns[['class', 'gender', 'age']].groupby(['class', 'gender']).aggregate('count').transpose()
kmeans_cython_centroids = np.array(sns_kmeans_cython.get_centroids())

In [15]:
for i in range(0, 9):
    print i, (df_sns['class'] == i).sum()
F_kmeans_cython = df_sns.drop(['gradyear', 'gender', 'age', 'friends', 'class'], axis=1)
F_kmeans_cython = pd.DataFrame(columns=F_kmeans_cython.columns, data=kmeans_cython_centroids).transpose()
for i in range(0, 9):
    print (F_kmeans_cython.loc[:,i]).sort_values()[-5:]

0 504
1 383
2 443
3 720
4 688
5 894
6 3824
7 16019
8 530
hot             0.199109
mall            0.220395
football        0.299542
shopping        0.485949
cheerleading    5.510765
Name: 0, dtype: float64
death     0.275681
church    1.217691
god       2.253207
jesus     2.369786
bible     6.153734
Name: 1, dtype: float64
rock          0.627325
sports        0.682258
basketball    0.763690
football      1.340572
baseball      5.492383
Name: 2, dtype: float64
drunk     1.939984
sex       2.138034
hair      2.605652
drugs     2.957622
kissed    3.168000
Name: 3, dtype: float64
clothes        0.554469
mall           0.623619
shopping       0.763631
abercrombie    3.917823
hollister      4.115273
Name: 4, dtype: float64
football      0.243619
sports        0.779573
basketball    0.943376
volleyball    2.674364
softball      3.056029
Name: 5, dtype: float64
dress       0.597157
mall        0.628572
dance       0.641483
cute        0.693184
shopping    0.837276
Name: 6, dtype: float64
death

In [16]:
from sklearn.cluster import KMeans

sns_kmeans_sklearn = KMeans(n_clusters=kluster_num, random_state=kmeans_rnd_seed)

In [17]:
%%timeit
sns_kmeans_sklearn.fit(sns_val_norm)

1 loop, best of 3: 3.57 s per loop


# Комментарии и пояснения
## Kmeans (python + numpy)
### Описание оптимальности работы:
* В программе присутствуют следующие циклы:  
    * for it in range(self.max_iter):  
        От него не возможно избавиться, т.к. это цикл определяет   
    количество выполняемых операций, а тело цикла не зависит   
    от переменной it  
    * for x in xrange(0, self.k): (встречается 2 раза)  
        Теоретичести, можно избавится от этого цикла путем замены на apply_along_axis  
    некоторой функции. Однако, с учетом того, что сам цикл выполняется крайне малое  
    число раз (O(кол-во центроидов)), а таккже, что применяемая функция не   
    раскладывается на встроенные функции  (т.к. присутствует условный переход) делать  
    такую замену бессмысленно.
* Определяя с помощью LineProfiler критичестие места программы видим:  
    * labels = self.predict(X) - ~55% процессорного времени  
        Метод self.predict использует только матричные операции. Здесь ускорять нечего
    * new_dev += np.apply_along_axis(...).sum() - ~25% процессорного времени
    * self.centroids[x] = X[self.labels == x].mean(0) - ~10% процессорного времени  
        Аналогично, здесь только матричные операции  

### Определение времени работы
* Используя %%timeit, получаем, что в среднем данная реализация работает за ~4.5 секунды (на моём устройстве)  

## Kmeans (cython)  

### Описание оптимальности работы:
* С помощью ключа -а определяем, какие участки кода плохо отображаются в C код cython  
    * Объявления функций
    * Выделение пямяти под массывы  
        На это не возможно повлиять.  

### Определение времени работы
* Используя %%timeit, получаем, что в среднем данная реализация работает за ~1.5 секунды (на моём устройстве)  

## Kmeans (sklearn)  
### Определение времени работы
* Используя %%timeit, получаем, что в среднем данная реализация работает за ~3.7 секунды (на моём устройстве) 

## Вывод
Получаем, что реализация с использованием numpy и векторизации по скорости довольно близка к скорости работы алгоритма из sklearn. В то же время, реализация на cython значительно превосходит по скорости обе вышеназванных реализации kmeans